In [1]:
!pip install pandas

DEPRECATION: o365 2.0.16 has a non-standard dependency specifier tzlocal<3.*,>=1.5.0. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of o365 or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import os

# Import ORDO data
ordo = pd.read_csv(os.path.join(os.getcwd(), 'rawdl_20240310', 'ordo.csv'))

In [3]:
# Filter the gene
ordo_gene = ordo[ordo['http://www.w3.org/2004/02/skos/core#notation'].isnull()]
ordo_gene = ordo[ordo['http://www.orpha.net/ORDO/symbol'].notnull()]

# Filter the disease
ordo_disease = ordo[ordo['http://www.w3.org/2004/02/skos/core#notation'].notnull()]

# Filter the definition
ordo_other = ordo[~ordo['Class ID'].isin(ordo_gene['Class ID']) & ~ordo['Class ID'].isin(ordo_disease['Class ID'])]

print('Gene : ', len(ordo_gene))
print('Disease : ', len(ordo_disease))
print('Other : ', len(ordo_other))
print('Total : ', len(ordo))

Gene :  4429
Disease :  10892
Other :  28
Total :  15349


In [4]:
# # View and Iterate the data in ORDO
# columns = ordo.columns

# for item in ordo.head().iterrows() :
#     print('====================')
#     for column in columns :
#         print(column, ' : ', item[1][column])

In [5]:
# Generate ORDO Disease / DIagnosis Map Subset
clean_ordo = pd.DataFrame()
clean_ordo['ORPHA'] = ordo_disease['http://www.w3.org/2004/02/skos/core#notation']
clean_ordo['label'] = ordo_disease['Preferred Label']
clean_ordo['Map'] = ordo_disease['http://www.geneontology.org/formats/oboInOwl#hasDbXref']

In [6]:
# Splitting the 'Map' column by '|'
split_maps = clean_ordo['Map'].str.split('|').tolist()
expanded_data = []

for i in range(len(split_maps)):
    # Check if split_maps is not nan or empty
    if type(split_maps[i])==float or len(split_maps[i])==0:
        continue
    else:
        for item in split_maps[i]:
            expanded_data.append([clean_ordo.iloc[i]['ORPHA'], clean_ordo.iloc[i]['label'], item])

# Creating a new DataFrame
expanded_ordo_map = pd.DataFrame(expanded_data, columns=['ORPHA', 'label', 'Map'])
expanded_ordo_map.head()

,ORPHA,label,Map
0,ORPHA:478664,Hereditary sensory and autonomic neuropathy ty...,ICD-10:G60.8
1,ORPHA:478664,Hereditary sensory and autonomic neuropathy ty...,UMLS:C4225308
2,ORPHA:478664,Hereditary sensory and autonomic neuropathy ty...,OMIM:616488
3,ORPHA:168,Loose anagen syndrome,MedDRA:10085066
4,ORPHA:168,Loose anagen syndrome,ICD-11:EC21.0


In [7]:
# List all type of ORPHA to external code mapping
expanded_ordo_map[['Term Type', 'Mapped Code']] = expanded_ordo_map['Map'].str.split(':', expand=True)
expanded_ordo_map['Term Type'].value_counts()

UMLS      9008
ICD-10    8495
OMIM      8199
ICD-11    5781
MeSH      3196
MedDRA    1601
Name: Term Type, dtype: int64

In [8]:
# Get ICD10, ORPHA, OMIM code mapping
orpha_icd10 = expanded_ordo_map[expanded_ordo_map['Term Type']=='ICD-10'][['ORPHA', 'Map']].drop_duplicates()
orpha_omim = expanded_ordo_map[expanded_ordo_map['Term Type']=='OMIM'][['ORPHA', 'Map']].drop_duplicates()

# Combine ordo_icd10 and ordo_omim left join on ORPHA (expanded_ordo_map), and coalesce the value
icd10_omim = pd.merge(orpha_omim, orpha_icd10, on='ORPHA', how='outer', suffixes=('OMIM', 'ICD10'))
icd10_omim = merged_ordo[merged_ordo['MapOMIM'].notnull() & merged_ordo['MapICD10'].notnull()][['MapICD10', 'MapOMIM']].drop_duplicates()

NameError: name 'merged_ordo' is not defined

In [ ]:
## Final Cleanse and Save the Data
### Rename the columns to match the standard Orpha, OMIM, ICD10

icd10_omim.columns = ['ICD10', 'OMIM']
icd10_omim.to_csv(os.path.join('subset', 'icd102omim_subset.tsv'), index=False, sep='\t')

orpha_omim.columns = ['ORPHA', 'OMIM']
orpha_omim.to_csv(os.path.join('subset', 'orpha2omim_subset.tsv'), index=False, sep='\t')